In [ ]:
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (6, 6)

import math
import numpy as np
import matplotlib.pyplot as plt

import scipy
import scipy.stats

from astropy.io import fits
import os.path

import sys
sys.path.append("../")
sys.path.append("../utils")

import datapipe
import common_functions as common


## Options

In [ ]:
!mr_filter -h

## Tool functions

### Afficher les informations d'un fichier fits

In [ ]:
# %load /Users/jdecock/git/pub/jdhp/snippets/python/astropy/fits/print_infos.py
def fits_info(file_path):
    hdu_list = fits.open(file_path)
    hdu_list.info()
    hdu_list.close()

### Créer un fichier fits

In [ ]:
# %load /Users/jdecock/git/pub/jdhp/snippets/python/astropy/fits/create_a_2d_fits_image_v2.py
def data_to_fits(img, file_path):
    hdu = fits.PrimaryHDU(img)
    hdu.writeto(file_path, clobber=True)

### Lire un fichier fits

In [ ]:
def fits_to_data(file_path):
    hdu_list = fits.open(file_path) # Open the FITS file
    data = hdu_list[0].data
    hdu_list.close()                # Close the FITS file
    return data

### Charger un fichier fits mrfilter

In [ ]:
def read_mr_file(file_path):
    hdu_list = fits.open(file_path) # Open the FITS file
    data = hdu_list[0].data
    hdu_list.close()                # Close the FITS file
    return data

### Plot

In [ ]:
def plot(data, title="", log=False):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)

    ax1.set_title(title)
    
    if log:
        # See http://matplotlib.org/examples/pylab_examples/pcolor_log.html
        #     http://stackoverflow.com/questions/2546475/how-can-i-draw-a-log-normalized-imshow-plot-with-a-colorbar-representing-the-raw
        #im = ax1.pcolor(x, y, image_array, norm=LogNorm(vmin=0.01, vmax=image_array.max()), cmap=self.color_map)  # TODO: "vmin=0.01" is an arbitrary choice...
        im = ax1.imshow(data, interpolation='nearest', origin='lower', norm=LogNorm(vmin=0.01, vmax=data.max()), cmap="gnuplot2")   # cmap=cm.inferno and cmap="inferno" are both valid
    else:
        im = ax1.imshow(data, interpolation='nearest', origin='lower', cmap="gnuplot2")   # cmap=cm.inferno and cmap="inferno" are both valid
        #im = ax1.pcolor(x, y, image_array, cmap=self.color_map, vmin=z_min, vmax=z_max)

    plt.colorbar(im, ax=ax1)  # draw the colorbar
    
    #ax2 = fig.add_subplot(212)
    #ax2.hist(data)
    
    plt.show()

In [ ]:
#datapipe.io.images.plot(img, title="...")
#common.plot_image_meter(ax, img, pix_pos, title="...", plot_log_scale=True)

## Experiments with mock data

### Make data

In [ ]:
size = 400

x = np.linspace(.0, np.pi, size)
y = np.linspace(.0, np.pi, size)
X, Y = np.meshgrid(x, y)

Z = np.sin(X*4.) * np.sin(Y*4.) + 1.

#Zmask = np.ones(Zn.shape, dtype=np.bool)
#border_size = int(size / 4)
#Zmask[border_size:2*border_size,border_size:2*border_size] = False
#Z[Zmask] = 0

Zn = Z + np.random.uniform(low=0., high=2., size=Z.shape)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Z, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Zn, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

In [ ]:
Zmask = np.zeros(Zn.shape, dtype=np.bool)

border_size = int(size / 4)
Zmask[:border_size,:border_size] = True
Zmask[-border_size:,-border_size:] = True
Zmask[:border_size,-border_size:] = True
Zmask[-border_size:,:border_size] = True

In [ ]:
Znc = Zn.copy()
Znc[Zmask] = 0

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Znc, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

In [ ]:
#Zm = np.logical_not(Zmask).astype(np.int8, copy=True)
#Zm = np.logical_not(Zmask).astype(np.int, copy=True)
Zm = np.logical_not(Zmask).astype(np.uint8, copy=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Zm, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

### Make a fits files for mr_filter

#### Noised image without corner

In [ ]:
data_to_fits(Zn, "zn.fits")
plot(fits_to_data("zn.fits"))

#### Noised image with corners

In [ ]:
data_to_fits(Znc, "znc.fits")
plot(fits_to_data("znc.fits"))

#### Mask

In [ ]:
data_to_fits(Zm, "mask.fits")
plot(fits_to_data("mask.fits"))

In [ ]:
fits_info("mask.fits")

### Reference

In [ ]:
plot(Z)

### Call mr_filter

In [ ]:
!mr_filter -w planes.mr -n4 zn.fits out.mr

fits_info("out.mr")
plot(read_mr_file("out.mr"))

plot(read_mr_file("planes.mr")[0])
plot(read_mr_file("planes.mr")[1])
plot(read_mr_file("planes.mr")[2])
plot(read_mr_file("planes.mr")[3])

In [ ]:
!mr_filter -w planes.mr -n4 znc.fits out.mr

fits_info("out.mr")
plot(read_mr_file("out.mr"))

plot(read_mr_file("planes.mr")[0])
plot(read_mr_file("planes.mr")[1])
plot(read_mr_file("planes.mr")[2])
plot(read_mr_file("planes.mr")[3])

In [ ]:
!mr_filter -w planes.mr -n4 -I mask.fits znc.fits out.mr
#!mr_filter -w planes.mr -n4 -K -I mask.fits znc.fits out.mr

fits_info("out.mr")
plot(read_mr_file("out.mr"))

plot(read_mr_file("planes.mr")[0])
plot(read_mr_file("planes.mr")[1])
plot(read_mr_file("planes.mr")[2])
plot(read_mr_file("planes.mr")[3])

## Experiments with mock data 2

### Make data

In [ ]:
size = 400

x = np.linspace(.0, np.pi, size)
y = np.linspace(.0, np.pi, size)
X, Y = np.meshgrid(x, y)

mu, sigma = np.pi/2., 8
Z = np.exp(-((X-mu)**2 + (Y-mu)**2) / 2.0 * sigma**2) * 100.

#Zmask = np.ones(Zn.shape, dtype=np.bool)
#border_size = int(size / 4)
#Zmask[border_size:2*border_size,border_size:2*border_size] = False
#Z[Zmask] = 0

Zn = Z + np.random.uniform(low=0., high=100., size=Z.shape)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Z, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Zn, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

In [ ]:
Zmask = np.zeros(Zn.shape, dtype=np.bool)
Zmask[150:250,150:250] = True

In [ ]:
Znc = Zn.copy()
Znc[Zmask] = 0

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Znc, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

In [ ]:
#Zm = np.logical_not(Zmask).astype(np.int8, copy=True)
#Zm = np.logical_not(Zmask).astype(np.int, copy=True)
Zm = np.logical_not(Zmask).astype(np.uint8, copy=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(Zm, interpolation='nearest', origin='lower', cmap="gray") # gnuplot2
plt.colorbar(im, ax=ax)  # draw the colorbar

plt.show()

### Make a fits files for mr_filter

#### Noised image without corner

In [ ]:
data_to_fits(Zn, "zn.fits")
plot(fits_to_data("zn.fits"))

#### Mask

In [ ]:
data_to_fits(Zm, "mask.fits")
plot(fits_to_data("mask.fits"))

In [ ]:
fits_info("mask.fits")

### Reference

In [ ]:
plot(Z)

### Call mr_filter

In [ ]:
!mr_filter -w planes.mr -n4 zn.fits out.mr

fits_info("out.mr")
plot(read_mr_file("out.mr"))

plot(read_mr_file("planes.mr")[0])
plot(read_mr_file("planes.mr")[1])
plot(read_mr_file("planes.mr")[2])
plot(read_mr_file("planes.mr")[3])

In [ ]:
!mr_filter -w planes.mr -n4 -I mask.fits zn.fits out.mr

fits_info("out.mr")
plot(read_mr_file("out.mr"))

plot(read_mr_file("planes.mr")[0])
plot(read_mr_file("planes.mr")[1])
plot(read_mr_file("planes.mr")[2])
plot(read_mr_file("planes.mr")[3])

## Experiment with ASTRI data

In [ ]:
#datapipe.io.images.plot(img, title="plot()")
#common.plot_image_meter(ax1, inp_img, pix_pos, title="Input image", plot_log_scale=True)

In [ ]:
#file_path = "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV07702.fits"
file_path = "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV07702.fits"

inp_ref = datapipe.io.images.load_benchmark_images(file_path)[0]["reference_image"]
inp_img = datapipe.io.images.load_benchmark_images(file_path)[0]["input_image"]
pix_pos = datapipe.io.images.load_benchmark_images(file_path)[0]["pixels_position"]
msk_img = datapipe.io.images.load_benchmark_images(file_path)[0]["pixels_mask"]

In [ ]:
!rm -v znc.fits

In [ ]:
data_to_fits(inp_img, "znc.fits")
plot(fits_to_data("znc.fits"))
#print(fits_to_data("znc.fits"))

In [ ]:
!rm -v mask.fits

In [ ]:
msk_img = msk_img.astype(np.uint8, copy=True)
data_to_fits(msk_img, "mask.fits")
plot(fits_to_data("mask.fits"))
#print(fits_to_data("mask.fits"))

In [ ]:
!rm -v out.mr planes.mr

In [ ]:
!mr_filter -w planes.mr -K -k -C1 -m3 -n4 -s4 out.mr
#!mr_filter -w planes.mr -K -k -C1 -m3 -n4 -s2,2,3,3 -I mask.fits znc.fits out.mr
#!mr_filter -w planes.mr -n4 -I mask.fits znc.fits out.mr

fits_info("out.mr")
plot(read_mr_file("out.mr"))

plot(read_mr_file("planes.mr")[0])
plot(read_mr_file("planes.mr")[1])
plot(read_mr_file("planes.mr")[2])
plot(read_mr_file("planes.mr")[3])